In [2]:
from os import curdir
from fastapi import APIRouter, HTTPException, UploadFile, File, Form
from typing import List
import json
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_community.vectorstores import FAISS
import tempfile


DB_INDEX = f"../backend/vectors/202508141516411874"
embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")
vector_db = FAISS.load_local(DB_INDEX, embeddings, allow_dangerous_deserialization=True)

retriever = vector_db.as_retriever(search_kwargs={"k": 5})



In [ ]:
system_prompt = """당신은 문서 기반 질의응답을 도와주는 AI 어시스턴트입니다. 
        제공된 문서 내용을 바탕으로 정확하고 유용한 답변을 제공하세요.
        문서에 없는 내용에 대해서는 "문서에서 해당 내용을 찾을 수 없습니다"라고 답변하세요.
        
        ### Context
        {docs}
        """
        
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{question}")
])
        
        

In [ ]:


message = "한신대학교 총장이 누구야?"
find_docs = retriever.invoke(message)
system_prompt = """당신은 문서 기반 질의응답을 도와주는 AI 어시스턴트입니다. 
        제공된 문서 내용을 바탕으로 정확하고 유용한 답변을 제공하세요.
        문서에 없는 내용에 대해서는 "문서에서 해당 내용을 찾을 수 없습니다"라고 답변하세요.
        
        ### Context
        {docs}
        """
        
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{question}")
])


llm = ChatOllama(model="gpt-oss:20b", temperature=0)

chain = rag_prompt | llm
        
response = chain.invoke({"question": message, "docs": find_docs})


ai_response = response.content

In [8]:
find_docs

[Document(id='0e4dd644-7876-486b-b9c4-f6339913893e', metadata={'producer': 'PDF 2022 12.0.0.4204', 'creator': 'PDF 2022 12.0.0.4204', 'creationdate': '2025-06-30T11:28:00+09:00', 'moddate': '2025-06-30T11:28:11+09:00', 'source': '/tmp/tmpq453ynbq.pdf', 'total_pages': 1000, 'page': 164, 'page_label': '165'}, page_content='2025한국디지털콘텐츠학회 하계종합학술대회 및 대학생논문경진대회 발표 논문집\n95\n후크 시스템을 활용한 입체적 전투 RPG개발장준혁(*),송유민(*),이은경(**),오석희(***)(*)한신대학교 컴퓨터공학부,{jhjang888,songdaldal}@hs.ac.kr(**)한신대학교 AIžSW대학,992cog22@hanmail.net(***)한신대학교 AIžSW대학,seokhee5@hs.ac.kr1.연구 배경코로나19 이후 디지털 콘텐츠 소비 증가와 함께 다양한 플랫폼을 통한 게임 확산이 가속화되었다[1]. 엘든링, 마비노기 모바일, 로스트아크 등의 성공은 RPG 장르가 플랫폼에 관계없이 높은 몰입도와 선호도를 이끌어내는 강점을 보여준다, 아래 그림 1과 같이 Steam 플랫폼 내 RPG 장르 점유율 또한 2020년 6.7%에서 2025년 7.8%로 증가하며 지속적인 성장세를 나타내고 있다.\n그림 1.Steam플랫폼 RPG장르 점유율(%)(2020~2025)[2]본 연구는 이러한 배경하에 RPG 장르의 핵심 요소인 보스 레이드 전투와 입체적 전투 시스템을 결합한 액션 RPG 게임 개발을 목표로 한다.2.연구 내용2.1구현 환경본 연구는 Unity 2022.3 LTS와 C#을 활용하여 개발하였으며 후크 시스템과 FSM 구조는 Unity의 컴포넌트를 기반 구조를 활용해 모듈화하여 구현하였다.')

In [4]:
len(find_docs)

5

In [5]:
find_docs[0]

Document(id='0e4dd644-7876-486b-b9c4-f6339913893e', metadata={'producer': 'PDF 2022 12.0.0.4204', 'creator': 'PDF 2022 12.0.0.4204', 'creationdate': '2025-06-30T11:28:00+09:00', 'moddate': '2025-06-30T11:28:11+09:00', 'source': '/tmp/tmpq453ynbq.pdf', 'total_pages': 1000, 'page': 164, 'page_label': '165'}, page_content='2025한국디지털콘텐츠학회 하계종합학술대회 및 대학생논문경진대회 발표 논문집\n95\n후크 시스템을 활용한 입체적 전투 RPG개발장준혁(*),송유민(*),이은경(**),오석희(***)(*)한신대학교 컴퓨터공학부,{jhjang888,songdaldal}@hs.ac.kr(**)한신대학교 AIžSW대학,992cog22@hanmail.net(***)한신대학교 AIžSW대학,seokhee5@hs.ac.kr1.연구 배경코로나19 이후 디지털 콘텐츠 소비 증가와 함께 다양한 플랫폼을 통한 게임 확산이 가속화되었다[1]. 엘든링, 마비노기 모바일, 로스트아크 등의 성공은 RPG 장르가 플랫폼에 관계없이 높은 몰입도와 선호도를 이끌어내는 강점을 보여준다, 아래 그림 1과 같이 Steam 플랫폼 내 RPG 장르 점유율 또한 2020년 6.7%에서 2025년 7.8%로 증가하며 지속적인 성장세를 나타내고 있다.\n그림 1.Steam플랫폼 RPG장르 점유율(%)(2020~2025)[2]본 연구는 이러한 배경하에 RPG 장르의 핵심 요소인 보스 레이드 전투와 입체적 전투 시스템을 결합한 액션 RPG 게임 개발을 목표로 한다.2.연구 내용2.1구현 환경본 연구는 Unity 2022.3 LTS와 C#을 활용하여 개발하였으며 후크 시스템과 FSM 구조는 Unity의 컴포넌트를 기반 구조를 활용해 모듈화하여 구현하였다.')